In [19]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import lib.zigzag as zz
import lib.bars as bars

In [20]:
data = pd.read_csv("..\..\Datasets\WDO@_M5-with-zz.csv")
# columns = ['time', 'open', 'high', 'low', 'close']
columns = ['time', 'open', 'high', 'low', 'close', 'zz', 'highs', 'lows']
data = data.loc[:, columns]
# data['time']=pd.to_datetime(data['time'], unit='ms')
data.tail()

,time,open,high,low,close,zz,highs,lows
96519,2021-04-05 17:35:00,5677.5,5685.0,5677.5,5685.0,NaN,NaN,NaN
96520,2021-04-05 17:40:00,5685.0,5685.5,5682.5,5684.0,NaN,NaN,NaN
96521,2021-04-05 17:45:00,5684.5,5686.0,5676.5,5679.5,5686.0,5686.0,NaN
96522,2021-04-05 17:50:00,5679.0,5680.5,5676.0,5677.0,5676.0,NaN,5676.0
96523,2021-04-05 17:55:00,5676.5,5679.0,5672.0,5672.0,NaN,NaN,NaN


In [21]:
# parsed_ds=data.loc[(data['time']>='2021-03-19 8:00:00')&(data['time']<='2021-03-19 19:00:00')]
parsed_ds=data.loc[(data['time']>='2021-03-22 09:00:00')&(data['time']<='2021-03-25 19:00:00')]
parsed_ds.reset_index(inplace=True, drop=True)
parsed_ds.head()

,time,open,high,low,close,zz,highs,lows
0,2021-03-22 09:00:00,5510.0,5518.0,5507.0,5516.0,NaN,NaN,NaN
1,2021-03-22 09:05:00,5516.5,5519.0,5511.0,5514.5,5519.0,5519.0,NaN
2,2021-03-22 09:10:00,5515.0,5515.5,5500.5,5500.5,NaN,NaN,NaN
3,2021-03-22 09:15:00,5500.5,5504.0,5498.0,5501.5,NaN,NaN,NaN
4,2021-03-22 09:20:00,5501.0,5513.0,5497.0,5510.5,5497.0,NaN,5497.0


In [11]:
zz, lows, highs = zig_zag(data,3)

In [5]:
parsed_ds=parsed_ds.assign(zz=zz,lows=lows,highs=highs)

In [27]:
ds=data.loc[(data['time']>='2021-03-19 09:00:00')&(data['time']<='2021-03-23 19:00:00')]

In [22]:
fig = go.Figure(data=[go.Candlestick(open=parsed_ds['open'],
                high=parsed_ds['high'],
                low=parsed_ds['low'],
                close=parsed_ds['close'])])

fig.add_trace(go.Scatter(
    x=parsed_ds.loc[(parsed_ds['zz'].notnull())].index,
    y=parsed_ds.loc[(parsed_ds['zz'].notnull())]['zz']
))


fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), #hide weekends
        dict(bounds=[19, 9], pattern="hour"), #hide hours outside of 9am-6pm
    ]
)
fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()

In [23]:
'''
Cria uma Series com o inicio de cada perna
contando que cada perna começa na primeira
barra com direção igual ao ZigZag ou doji.

A perna na coluna leg_start pode 
terminar uma linha antes exluindo a ultima 
barra da perna. Logo a maneira correta de 
achar a perna completa é verificar da primeira
barra da perna até a próxima não nula ocorrencia
da coluna "zz".

É preciso saber o começo e o fim da perna para
extrair a features
'''


def zz_direction(row):
    if row['zz']!=np.nan:
        if row["zz"]==row["highs"] :
            return -1
        else:
            return 1
    return 0

# Versão lenta:
# def leg_start(_parsed_ds):
#     leg_start=np.full(_data.shape[0], np.nan)
#     leg_counter=0
#     for idx in range(_data.shape[0]):
#         if not np.isnan(_data.iloc[idx]["zz"]):
#             zz_direc=zz_direction(_data.iloc[idx])
#             found_start_bar=False
#             forward_idx=0
#             while not found_start_bar and idx+forward_idx<_data.shape[0]:
#                 bar_dir = bars.bar_direction(_data.iloc[idx+forward_idx])
#                 if zz_direc == bar_dir or \
#                     0 == bar_dir:
#                     leg_start[idx+forward_idx]=leg_counter
#                     leg_counter+=1
#                     found_start_bar=True
#                 forward_idx+=1
#     return leg_start

# def leg_start(_data):
#     leg_start=np.full(_data.shape[0], np.nan)
#     leg_start_direction=0
#     leg_counter=0
#     for idx, row in _data.iterrows():
#         bar_dir=bars.bar_direction(row)
#         if not np.isnan(row['zz']):
#             leg_start_direction=zz_direction(row)
#         if leg_start_direction!=0 and (bar_dir==leg_start_direction or bar_dir==0):
#             leg_start[idx]=leg_counter
#             leg_counter+=1
#             leg_start_direction=0
#     return leg_start

def leg_start2(row):
    leg_start_direction=0
    leg_counter=0
    bar_dir=bars.bar_direction(row)
    if not np.isnan(row['zz']):
        leg_start_direction=zz_direction(row)
    if leg_start_direction!=0 and (bar_dir==leg_start_direction or bar_dir==0):
        leg_counter+=1
        leg_start_direction=0
        return leg_counter

class LegStart():
    leg_start_direction=0
    leg_counter=0

    @classmethod
    def leg_start(cls,row):
        bar_dir=bars.bar_direction(row)
        if not np.isnan(row['zz']):
            cls.leg_start_direction=zz_direction(row)
        if cls.leg_start_direction!=0 and (bar_dir==cls.leg_start_direction or bar_dir==0):
            cls.leg_counter+=1
            cls.leg_start_direction=0
            return cls.leg_counter
        return np.nan


parsed_ds['leg_start']=parsed_ds.apply(lambda row: LegStart.leg_start(row), axis=1)
# parsed_ds=parsed_ds.assign(leg_start=leg_start2(parsed_ds))
# parsed_ds.head(30)

C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_25752/1099185087.py:86: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [24]:
parsed_ds.head(20)

,time,open,high,low,close,zz,highs,lows,leg_start
0,2021-03-22 09:00:00,5510.0,5518.0,5507.0,5516.0,NaN,NaN,NaN,NaN
1,2021-03-22 09:05:00,5516.5,5519.0,5511.0,5514.5,5519.0,5519.0,NaN,1.0
2,2021-03-22 09:10:00,5515.0,5515.5,5500.5,5500.5,NaN,NaN,NaN,NaN
3,2021-03-22 09:15:00,5500.5,5504.0,5498.0,5501.5,NaN,NaN,NaN,NaN
4,2021-03-22 09:20:00,5501.0,5513.0,5497.0,5510.5,5497.0,NaN,5497.0,2.0
5,2021-03-22 09:25:00,5511.0,5518.5,5510.5,5516.0,NaN,NaN,NaN,NaN
6,2021-03-22 09:30:00,5516.0,5520.0,5511.5,5518.0,NaN,NaN,NaN,NaN
7,2021-03-22 09:35:00,5518.0,5532.0,5516.5,5531.0,NaN,NaN,NaN,NaN
8,2021-03-22 09:40:00,5531.0,5544.0,5530.0,5542.5,NaN,NaN,NaN,NaN
9,2021-03-22 09:45:00,5542.5,5545.0,5535.5,5539.0,NaN,NaN,NaN,NaN


In [ ]:
# Verificar se o idx está no começo da perna
# Passar um loop até o final
# O final é o primeiro não nulo após o começo da perna
# Atualizar a variavel começo_perna para o final_da_perna 
# leg_start_idx+leg_end_idx

# Versão Lenta:
# def iter_legs(_data, feature_fn=None):
#     leg_start_idx=0
#     while leg_start_idx < _data.shape[0]-1:
#         if np.isnan(_data.iloc[leg_start_idx]['leg_start']):
#             leg_start_idx+=1
#             continue
#         else:
#             for leg_end_idx in range(1,_data.shape[0]):
#                 if not np.isnan(_data.iloc[leg_start_idx+leg_end_idx]['zz']):
#                     if leg_end_idx>0:
#                         leg_start_idx+=leg_end_idx
#                     else:
#                         leg_start_idx+=1
#                     break

'''
Toda perna termina antes da proxima começar.
O if de achar o começo vem por ultimo porque
se ele achar o começo da perna a procura do 
final dela tem que vir com o leg_start_idx+1
'''

'''
A feature_fn sempre retorna um array do tamanho
do numero de pernas
'''
# def iter_legs(_data, feature):
#     feature_arr=[]
#     leg_start_idx=None
#     for idx, row in _data.iterrows():
#         if leg_start_idx!=None:
#             feature.run(leg_start_idx,idx)
            
#         if leg_start_idx!=None and not np.isnan(row['zz']):
#             leg_start_idx=None
#             feature_arr.append(feature.result())


#         if not np.isnan(row['leg_start']):
#             leg_start_idx=idx
#     return feature_arr


# print("arr: ", iter_legs(data, bar_countage()))



In [40]:
'''
FEATURE: Contagem de barras
Diferença entre leg_sart_idx and leg_end_idx
'''

class leg_reversal_attempts():
    def __init__(self) -> None:
        super().__init__()
        

In [41]:
b=directional_bar_count()
random_variable=parsed_ds.apply(lambda row: b.loop(row), axis=1)
random_variable=random_variable.loc[~np.isnan(random_variable)]
# print()
random_variable[:30]

4      2.0
11     6.0
16     5.0
21     5.0
29     4.0
31     1.0
36     4.0
44     8.0
47     2.0
54     7.0
61     6.0
66     4.0
72     5.0
77     5.0
84     6.0
87     2.0
99     9.0
107    7.0
111    3.0
120    7.0
122    2.0
124    3.0
128    4.0
134    4.0
138    5.0
146    7.0
152    5.0
156    4.0
165    8.0
168    4.0
dtype: float64

In [42]:
fig = go.Figure(data=[go.Candlestick(x=parsed_ds['time'],
                open=parsed_ds['open'],
                high=parsed_ds['high'],
                low=parsed_ds['low'],
                close=parsed_ds['close'])])

# fig.add_trace(go.Scatter(
#     x=data.loc[(data['zz'].notnull())]['time'],
#     y=data.loc[(data['zz'].notnull())]['zz']
# ))

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), #hide weekends
        dict(bounds=[18, 9], pattern="hour"), #hide hours outside of 9am-6pm
    ]
)

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()

In [6]:
# Perna total expandida comparada com
# a perna como ela aparece no grafico.
# Relação continua de sobreposição entre 
# as barras.

contracted_leg = parsed_ds.iloc[parsed_ds.shape[0]-1]['high']-parsed_ds.iloc[0]['low']
expanded_leg=0
for idx in range(parsed_ds.shape[0]):
    expanded_leg+=parsed_ds.iloc[idx]['high']-parsed_ds.iloc[idx]['low']

print('expanded_leg: {}, contracted_leg: {}'.format(expanded_leg,contracted_leg))

expanded_leg: 93.0, contracted_leg: 32.5


In [7]:
leg_direction = -1

In [8]:
'''
Bars that trade below 50% of the bar before
'''

overlap_count=0
for idx in range(data.shape[0]):
    if idx <= 1:
        continue
    fifty=data.iloc[idx-1]['low']+(data.iloc[idx-1]['high']-data.iloc[idx-1]['low'])/2
    if leg_direction > 0 and fifty > data.iloc[idx]['low']:
        overlap_count+=1
    if leg_direction < 0 and fifty < data.iloc[idx]['high']:
        overlap_count+=1
    
    # print(idx)


In [18]:
def between(value, bar, body=True):
    direction = bars.bar_direction(bar)
    if body:
        if direction>0:
            if bar['close']>value and bar['open'] < value:
                return True
        if direction<0:
            if bar['close']<value and bar['open'] > value:
                return True
    else:
        if bar['high']>value and bar['low'] < value:
            return True
    return False

def inside_bar(outside, inside, inside_body=False):
    if not inside_body:
        if outside['high']>=inside['high'] and outside['low']<=inside['low']:
            return True
    else:
        direction=bars.bar_direction(inside)
        if direction>0:
            if outside['high']>=inside['close'] and outside['low']<=inside['open']:
                return True
        if direction<0:
            if outside['high']>=inside['open'] and outside['low']<=inside['close']:
                return True
    return False

In [57]:
inside_bar(parsed_ds.iloc[6],parsed_ds.iloc[7]) or \
inside_bar(parsed_ds.iloc[6],parsed_ds.iloc[7], inside_body=True) or \
between(parsed_ds.iloc[6]['close'],parsed_ds.iloc[7]) or \
between(parsed_ds.iloc[6]['open'],parsed_ds.iloc[7])

True

In [63]:
'''
Bars that overlap with the body of bars before
- The first bar of the range does not count
- The body overlaps with two bars before or more
'''

overlaping_count=bar_before=last_overlaping=0
range_starts=[]
while bar_before<parsed_ds.shape[0]-2:
    for overlap_bar in range(bar_before+2, parsed_ds.shape[0]):
        doji=bars.doji(parsed_ds.iloc[bar_before])
        if doji:
            overlap_flag = inside_bar(parsed_ds.iloc[bar_before],parsed_ds.iloc[overlap_bar]) or \
                        inside_bar(parsed_ds.iloc[bar_before],parsed_ds.iloc[overlap_bar], inside_body=True) or \
                        between(parsed_ds.iloc[bar_before]['high'],parsed_ds.iloc[overlap_bar]) or \
                        between(parsed_ds.iloc[bar_before]['low'],parsed_ds.iloc[overlap_bar])
            if overlap_bar == bar_before+2:
                if not overlap_flag:
                    break
            if overlap_flag:
                overlaping_count+=1
                last_overlaping=overlap_bar
                # print('bar_before: ', bar_before)
        if not doji:
            overlap_flag=inside_bar(parsed_ds.iloc[bar_before],parsed_ds.iloc[overlap_bar]) or \
                    inside_bar(parsed_ds.iloc[bar_before],parsed_ds.iloc[overlap_bar], inside_body=True) or \
                    between(parsed_ds.iloc[bar_before]['close'],parsed_ds.iloc[overlap_bar]) or \
                    between(parsed_ds.iloc[bar_before]['open'],parsed_ds.iloc[overlap_bar])
            if overlap_bar == bar_before+2:
                if not overlap_flag:
                    break
            if overlap_flag:
                overlaping_count+=1
                last_overlaping=overlap_bar

    if last_overlaping>bar_before:
        # print('last_overlaping: {} bar_before: {}'.format(last_overlaping,bar_before))
        range_starts.append(bar_before)
        bar_before=last_overlaping
    else:
        bar_before+=1
        # print('bar_before: ', bar_before)

overlaping_count+=len(range_starts)
print('overlaping_count: ', overlaping_count)
print('range_starts: ', range_starts)

last_overlaping: 4 bar_before: 0
last_overlaping: 10 bar_before: 6
overlaping_count:  8
range_starts:  [0, 6]
